Загрузим библиотеку fpgen

In [1]:
from IPython.display import clear_output

! pip install git+https://github.com/rimgro/biocadprotein/fpgen.git

clear_output()

In [20]:
from fpgen.generator import ProteinGenerator
from fpgen.metrics import CustomMetric

from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein
from esm.utils.structure.protein_chain import ProteinChain

In [ ]:
# GFP
gfp_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('1qy3', chain_id='A')
)
gfp_aplha_phelix = ...

# mScarlet
mscarlet_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('5lk4', chain_id='A')
)
mscarlet_aplha_phelix = ...

# sfGFP
sfgfp_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('2b3p', chain_id='A')
)
sfgfp_aplha_phelix = ...

Загрузим из базы данных PBD белок GFP и создадим объект генератора

In [ ]:
model = ESM3.from_pretrained('esm-open').to('cuda')

gfp_generator = ProteinGenerator(gfp_protein, gfp_aplha_phelix, model)

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\esm\models\vqvae.py:286: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):  # type: ignore


В библиотеке уже есть заготовленная функция для подсчета RMSD по всей структуре белка. Но нет для подсчета по альфа-спирали (по техническим причинам). Поэтому ее необходимо реализовать самостоятельно

In [21]:
def phelix_rmsd(generation_protein, template_protein, alpha_phelix) -> float:
    template_chain: ProteinChain = template_protein.to_protein_chain()
    generation_chain: ProteinChain = generation_protein.to_protein_chain()

    rmsd = template_chain[alpha_phelix].rmsd(generation_chain[alpha_phelix])
    return rmsd

In [ ]:
import os
os.environ['DISABLE_ITERATIVE_SAMPLING_TQDM'] = '1'

In [ ]:
from tqdm import trange

In [ ]:
metrics = []
proteins = []

In [ ]:
for _ in trange(1000):
    protein, metric = gfp_generator.generate(
        metric_list=[
            CustomMetric(phelix_rmsd, gfp_aplha_phelix),
            'rmsd',
            'ptm',
            'plddt'
        ]
    )

    metrics.append(metric)
    proteins.append(protein)

100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


In [ ]:
import pandas as pd
metrics_df = pd.DataFrame(metrics, columns=['alpha_rmsd', 'full_rmsd', 'ptm', 'plddt'])